In [1]:
from os.path import join
import numpy as np
import pandas as pd
#from make_dataset import make_dataset
from sklearn.metrics import mean_absolute_error
import torch
import torch.nn as nn
import torch.optim as optim

<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [2]:
import pandas as pd
import numpy as np
import glob
import matplotlib.pyplot as plt
import seaborn as sns
from filterpy.kalman import KalmanFilter
from filterpy.common import Q_discrete_white_noise
from tqdm import tqdm
from sklearn.decomposition import PCA


def preprocessing1(input_path):
  data = pd.read_csv('\\Users\\ineeji\\Desktop\\past\\datas\\데이터합본_파생변수 제거.csv')

  data['year'] = data['Unnamed: 0'].apply(lambda x : x.split()[0].split('-')[0])
  data['month'] = data['Unnamed: 0'].apply(lambda x : x.split()[0].split('-')[1])
  data['date'] = data['Unnamed: 0'].apply(lambda x : x.split()[0].split('-')[2])
  data['hour'] = data['Unnamed: 0'].apply(lambda x : x.split()[1].split(' ')[0].split(':')[0])

  data['year'] = data['year'].astype('int')
  data['month'] = data['month'].astype('int')
  data['date'] = data['date'].astype('int')
  data['hour'] = data['hour'].astype('int')
  return data

def drop_under_TI_360(data):
   data = data[data['TI21022A(Catalyst T 1)'] > 370]
   return data

def Tree_sigma(df):
  lower_out = df['DSL D-95'].mean() - df['DSL D-95'].std()*3
  #upper_out = df['DSL D-95'].mean() + df['DSL D-95'].std()*3
  #df = df[(df['DSL D-95'] > lower_out) & (df['DSL D-95'] < upper_out)]
  return df


def drop_List(df,train,test, n):
  dl = df.corr()['DSL D-95'].map(abs).sort_values(ascending = False)[10:]
  #dl = dl.drop('DSL D-95')
  dl = pd.DataFrame(dl)
  dl = dl.reset_index()
  dl = dl['index']
  #dl = dl.drop('DSL D-95')
  #print(dl)
  train.drop(dl,axis=1,inplace=True)
  test.drop(dl,axis=1,inplace=True)
  return train,test


def preprocessing2(df):
  train = df[(df['year'] == 2015) |(df['year'] == 2016) | (df['year'] == 2017) | (df['year'] == 2018)]
  test = df[(df['year'] == 2019) |(df['year'] == 2020) | (df['year'] == 2021)]
  return train, test

def drop_abnormal_region(df, target_col, std_threshold, rate_threshold):
    """
    데이터프레임에서 타겟 컬럼이 갑자기 떨어지는 구간을 찾아 해당 구간을 드랍시킵니다.
    """
    # 감소 구간 탐색을 위한 변수 초기화
    is_dropping = False
    start_idx = None
    
    # 드랍시킬 구간을 기록할 리스트 초기화
    drop_regions = []
    
    # 타겟 컬럼의 표준편차와 변화량에 대한 기준값 계산
    std = df[target_col].std()
    threshold = std * std_threshold
    #print(std,threshold)
    #print(df.loc[330:340.:])
    # 전체 데이터에 대해 반복
    for i in range(5, len(df)):
        # 이전 값과의 차이를 계산
        diff = abs(df[target_col][i] - df[target_col][i-5])
        
        # 감소 구간 탐색
        if diff / std >= rate_threshold:
            start_idx = i - 5
            #print("========================================")
            #print(df[target_col][i],df[target_col][i-5], i, i - 5)
            #print(diff, std, rate_threshold, start_idx,i-1)
            #print("========================================")
            for j in range(start_idx,i):
                drop_regions.append(j)
            #drop_regions.append((start_idx, i - 1))
            #is_dropping = True
        #elif is_dropping and diff / std > rate_threshold:
            #print(start_idx,i-1)
            #drop_regions.append((start_idx, i - 1))
            #is_dropping = False
    #print(drop_regions)
    # 드랍시킬 구간을 데이터프레임에서 제거
    print(drop_regions)
    '''
    for start, end in drop_regions:
        df = df.drop(df.index[start:end+1])'''
    df = df.drop(drop_regions, axis=0)
    #print(df.loc[5:15.:])
    return df

'''
def drop_abnormal_region(df, target_col, std_threshold, rate_threshold):
    """
    데이터프레임에서 타겟 컬럼이 갑자기 떨어지는 구간을 찾아 해당 구간을 드랍시킵니다.
    """
    # 감소 구간 탐색을 위한 변수 초기화
    is_dropping = False
    start_idx = None
    
    # 드랍시킬 구간을 기록할 리스트 초기화
    drop_regions = []
    
    # 타겟 컬럼의 표준편차와 변화량에 대한 기준값 계산
    std = df[target_col].std()
    threshold = std * std_threshold
    
    # 전체 데이터에 대해 반복
    for i in range(1, len(df)):
        # 이전 값과의 차이를 계산
        diff = abs(df[target_col][i] - df[target_col][i-1])
        
        # 감소 구간 탐색
        if not is_dropping and diff / std <= rate_threshold:
            start_idx = i - 1
            is_dropping = True
        elif is_dropping and diff / std > rate_threshold:
            drop_regions.append((start_idx, i-1))
            is_dropping = False
    
    # 드랍시킬 구간을 데이터프레임에서 제거
    for start, end in drop_regions:
        df = df.drop(df.index[start:end+1])
    
    return df
'''

def preprocessing3(train, test):
  train_x = train.drop(['DSL D-95'],axis=1)
  train_y = train['DSL D-95']
  test_x = test.drop(['DSL D-95'],axis=1)
  test_y = test['DSL D-95']
  #train_x.drop(['Unnamed: 0'],axis=1,inplace=True)
  #test_x.drop(['Unnamed: 0'],axis=1,inplace=True)
  return train_x, train_y, test_x, test_y


def plus_PCA(train_X, test_X, threshold=0.8, n_components=1):
    """
    feature들 간의 상관관계가 높은 feature들을 뽑아서 PCA를 시도한 후, 결과로 나온 피처를 추가하는 함수
    
    Args:
    train_X (pd.DataFrame): 학습 데이터셋 feature
    test_X (pd.DataFrame): 테스트 데이터셋 feature
    threshold (float): 상관계수 threshold (default: 0.8)
    n_components (int): PCA 컴포넌트 수 (default: 1)
    
    Returns:
    train_X_new (pd.DataFrame): 학습 데이터셋에 PCA 피처가 추가된 데이터프레임
    test_X_new (pd.DataFrame): 테스트 데이터셋에 PCA 피처가 추가된 데이터프레임
    """
    # 상관계수 행렬 계산
    corr_matrix = train_X.corr().abs()
    
    # threshold 이상의 상관계수를 가지는 feature들 추출
    high_corr_features = set()
    for i in range(len(corr_matrix.columns)):
        for j in range(i):
            if corr_matrix.iloc[i, j] >= threshold:
                high_corr_features.add(corr_matrix.columns[i])
                high_corr_features.add(corr_matrix.columns[j])
    high_corr_features = list(high_corr_features)
    
    # 추출된 feature들을 이용해 PCA를 적용하여 새로운 feature 생성
    pca = PCA(n_components=n_components)
    pca.fit(train_X[high_corr_features])
    pca_features_tr = pca.transform(train_X[high_corr_features])
    pca_features_te = pca.transform(test_X[high_corr_features])
    
    # 새로운 feature 추가
    train_X_new = train_X.copy()
    test_X_new = test_X.copy()
    for i in range(n_components):
        train_X_new[f"PCA_{i+1}"] = pca_features_tr[:, i]
        test_X_new[f"PCA_{i+1}"] = pca_features_te[:, i]
    train_X_new.drop(high_corr_features, axis = 1, inplace=True)
    test_X_new.drop(high_corr_features, axis = 1, inplace=True)
    return train_X_new, test_X_new



def filtering_7H(df):
  df = df[df['hour'] == 7]
  df = df.reset_index(drop = True)
  return df

def feature_selection(df):
  df['A+B'] = df['TI21022A(Catalyst T 1)'] + df['TI21022B(Catalyst T 2)']
  df = df[['PIC23106(Top P)', 'FIC23105(P/A RT Flow)', 'A+B','TI23118(D/O Liquid T)', 'TI23502(D/O Vapor T)']]
  return df


def drop_day_count(train_x,test_x):
  train_x.drop(['year','month','date','hour'],axis=1,inplace=True)
  test_x.drop(['year','month','date','hour'],axis=1,inplace=True)
  train_x.drop(['Unnamed: 0'],axis=1,inplace=True)
  test_x.drop(['Unnamed: 0'],axis=1,inplace=True)
  return train_x,test_x

def log(df):
  col_list = df.columns
  for i in col_list:
    df.loc[:,i + 'log'] = np.log1p(df.loc[:,i])
  return df

def kalman_filter(df):
  col_list = df.columns
  
  for i in tqdm(col_list):
      current=0
      sum_c=[]
      z = df.loc[:, i]
      a = []           #필터링 된 피쳐(after)
      b = []           #필터링 전 피쳐(before)
      my_filter = KalmanFilter(dim_x=2,dim_z=1) #create kalman filter
      my_filter.x = np.array([[2.],[0.]])       # initial state (location and velocity)
      my_filter.F = np.array([[1.,1.], [0.,1.]])    # state transition matrix
      my_filter.H = np.array([[1.,0.]])    # Measurement function
      my_filter.P *= 1000.                 # covariance matrix
      my_filter.R = 5                      # state uncertainty
      my_filter.Q = Q_discrete_white_noise(dim = 2,dt=.1,var=.1) # process uncertainty   
      for k in z.values:
          my_filter.predict()
          my_filter.update(k)
          # do something with the output
          x = my_filter.x
          a.extend(x[0])
          b.append(k)
      sum_c=sum_c+a
      df.loc[:,'kf_X_'+str(i)]=sum_c
  return df

def mean_median_30(df):
  raw_cols_tr = df.columns

  mean_arr = []
  median_arr = []


  for column in raw_cols_tr:
      column_list = df[column].to_list()
      for i in range(30):
        mean_arr.append(column_list[i])
        median_arr.append(column_list[i])
          
      for i in range(30, len(column_list)):
          mean_arr.append(float(np.mean(column_list[i-30:i])))
          median_arr.append(float(np.median(column_list[i-30:i])))
      df[f'{column}_mean_30'] = mean_arr
      df[f'{column}_median_30'] = median_arr
      mean_arr = []
      median_arr = []
  return df





In [7]:
from glob import glob
#from preprocessing import *

def make_dataset(input_path):

    pd.options.display.float_format = '{: .20f}'.format

    data = preprocessing1(input_path)
    


    #TI21022A 360 이하 드랍
    #data = drop_under_TI_360(data)
    #print(len(data[data['TI21022A(Catalyst T 1)'] <= 370]))
    #under만
    #data = Tree_sigma(data)

    

    train, test = preprocessing2(data)

    train = filtering_7H(train)
    test = filtering_7H(test)
    print(len(test))  
    zz = pd.concat([train,test],axis = 0)
    zz = zz.reset_index(drop = True)
    train = zz.loc[:1448,:]
    test = zz.loc[1449:,:]
    print(test)
    test = test.reset_index(drop = True)

    print(len(test))
    #train = drop_abnormal_region(train, 'DSL D-95', std_threshold=1, rate_threshold=0.5)
    test = drop_abnormal_region(test, 'DSL D-95', std_threshold=1, rate_threshold=0.5)
    train = train.reset_index(drop = True)
    test = test.reset_index(drop = True)
    print(len(test))




    train_x, train_y, test_x, test_y = preprocessing3(train,test)

    train_x, test_x = plus_PCA(train_x,test_x)

    # '+'상관관계 애들
    #train_x = train_x[['TIC23115(Feed1 T)','TI23029(Feed2 T)','TI23028(P/A RT T)','TI23120(F Zone T)','TI23121(SS T)','TI23502(D/O Vapor T)','TI23122(BTM T)','TI23119(OV T)','TI23118(D/O Liquid T)','FIC23133(R/D Flow)']]
    #test_x = test_x[['TIC23115(Feed1 T)','TI23029(Feed2 T)','TI23028(P/A RT T)','TI23120(F Zone T)','TI23121(SS T)','TI23502(D/O Vapor T)','TI23122(BTM T)','TI23119(OV T)','TI23118(D/O Liquid T)','FIC23133(R/D Flow)']]

    # EDA 기반
    #train_x = train_x[['FIC21192(F2 Flow)','TI23502(D/O Vapor T)','FIC23010(D/O Flow)','FIC25103(OVHD Flow 1)','FIC23133(R/D Flow)','FIC23110(OVHD Flow 2)']]
    #test_x = test_x[['FIC21192(F2 Flow)','TI23502(D/O Vapor T)','FIC23010(D/O Flow)','FIC25103(OVHD Flow 1)','FIC23133(R/D Flow)','FIC23110(OVHD Flow 2)']]

    
    #drop_list of corr
    #corr 상위 몇개만 사용할 지
    #14->15 될때 mae 급 상승(9->11)
    #18->19 될때 mae 급 감소(11->9)
    n = 10
    #train_x, test_x = drop_List(data,train_x,test_x, n)

    #train = filtering_7H(train)
    '''
    test = filtering_7H(test)
    '''
    
    #train_x, train_y, test_x, test_y = preprocessing3(train,test)



    #train_x = feature_selection(train_x)
    #test_x = feature_selection(test_x)

    #sns.boxplot(train_x)
    #sns.boxplot(test_x)


    ####Feature Engineering####

    train_x,test_x = drop_day_count(train_x,test_x)

    #train_x = log(train_x)
    #test_x = log(test_x)
    
    #train_x = kalman_filter(train_x)
    #test_x = kalman_filter(test_x)
    
    #train_x = mean_median_30(train_x)
    #test_x = mean_median_30(test_x)

    print("==================================")
    print("==================================")
    print("=============complete=============")
    print("==================================")
    print("==================================")
    return train_x, train_y, test_x, test_y
path = '\\Users\\ineeji\\Desktop\\past\\datas\\데이터합본_파생변수 제거.csv'

train_x, train_y, test_x, test_y = make_dataset(path)

920
           Unnamed: 0                  DSL D-95         FIC21185(F1 Flow)  \
1449  2019-01-01 7:00  399.19999999999998863132  646.13030000000003383320   
1450  2019-01-02 7:00  396.00000000000000000000  647.10440000000005511538   
1451  2019-01-03 7:00  396.19999999999998863132  646.89419999999995525286   
1452  2019-01-04 7:00  399.80000000000001136868  647.45090000000004692993   
1453  2019-01-05 7:00  401.00000000000000000000  647.31470000000001618901   
...               ...                       ...                       ...   
2364  2021-07-08 7:00  368.60000000000002273737  538.22289999999998144631   
2365  2021-07-09 7:00  369.39999999999997726263  536.85419999999999163265   
2366  2021-07-10 7:00  368.80000000000001136868  562.31640000000004420144   
2367  2021-07-11 7:00  366.39999999999997726263  561.63559999999995397957   
2368  2021-07-12 7:00  364.00000000000000000000  558.42870000000004893081   

             FIC21192(F2 Flow)         TIC23115(Feed1 T)  \
1449   44.8

In [4]:
from pycaret.regression import *

In [32]:
test_xy

,TI23123(Reflux T),FIC23104(Reflux F),FI23116(T-AR Flow 1),PCA_1,DSL D-95
0,40.58979999999999677129,85.94440000000000168257,181.88419999999999276952,-109.91943613526235878908,399.19999999999998863132
1,41.07770000000000010232,88.32359999999999899956,184.87100000000000932232,-110.63994712417853349962,396.00000000000000000000
2,40.93070000000000163709,86.24490000000000122782,180.68860000000000809450,-112.77102320655370704117,396.19999999999998863132
3,41.44200000000000017053,84.48940000000000338787,187.20099999999999340616,-113.56793137601921728219,399.80000000000001136868
4,41.40319999999999822649,83.89119999999999777174,182.64680000000001314220,-112.64428924525407182955,401.00000000000000000000
...,...,...,...,...,...
694,46.55499999999999971578,96.43049999999999499778,130.91599999999999681677,-171.30531588673102305620,368.60000000000002273737
695,46.32289999999999707825,93.60769999999999413376,130.55219999999999913598,-170.68364566931970216501,369.39999999999997726263
696,46.40319999999999822649,88.25350000000000250111,156.57349999999999567990,-182.04897586895054928391,368.80000000000001136868
697,46.74179999999999779448,77.46460000000000434284,166.55000000000001136868,-178.77326441490012598479,366.39999999999997726263


In [11]:
train_xy.describe()

,TI23123(Reflux T),FI23107(OF Flow),FIC23104(Reflux F),FI23116(T-AR Flow 1),FI23117(T-AR Flow 2),PCA_1,DSL D-95
count,1449.00000000000000000000,1449.00000000000000000000,1449.00000000000000000000,1449.00000000000000000000,1449.00000000000000000000,1449.00000000000000000000,1449.00000000000000000000
mean,43.53172608695656009559,28.76995203588681704332,84.17910186335404887359,126.90705672877852805414,274.32282836438974982229,-0.00000000000003597092,401.91038647342918466165
std,10.33146361618149988715,9.73303871210940130254,23.16741726782158394826,75.97650375301736858091,101.61921064283652071936,300.54479324148684327156,7.63072760987181641923
min,3.70179999999999997939,0.00000000000000000000,0.00000000000000000000,0.00000000000000000000,0.00000000000000000000,-174.48265373710546555230,364.80000000000001136868
25%,43.14430000000000120508,24.84629999999999938609,74.17610000000000525233,67.16049999999999897682,236.10140000000001236913,-126.05503494746699288953,400.00000000000000000000
50%,45.16340000000000287628,28.58699999999999974420,86.96219999999999572537,110.69459999999999411102,303.00130000000001473381,-90.42187539157662001799,403.80000000000001136868
75%,46.11670000000000158025,31.24849999999999994316,96.09780000000000654836,178.31749999999999545253,346.42020000000002255547,-35.52241258087009612154,406.39999999999997726263
max,131.67050000000000409273,75.14939999999999997726,179.25780000000000313776,573.36140000000000327418,405.25799999999998135536,1165.68773561626608170627,418.80000000000001136868


In [10]:
import pycaret
print(pycaret.__version__)

3.0.0


In [9]:
test_xy.describe()

,TI23123(Reflux T),FI23107(OF Flow),FIC23104(Reflux F),FI23116(T-AR Flow 1),FI23117(T-AR Flow 2),PCA_1,DSL D-95
count,699.00000000000000000000,699.00000000000000000000,699.00000000000000000000,699.00000000000000000000,699.00000000000000000000,699.00000000000000000000,699.00000000000000000000
mean,44.35195092989986420662,1.82896008583691038751,79.92575164520731334505,147.89473133047218311731,279.43024892703840578179,-57.87407093213325026682,395.21373390557943139356
std,3.09272601463732454974,6.96769732148742182432,12.75579933310439351146,66.57167474057152389832,65.29026367632548044639,64.28184459949783047250,15.43722792317374192805
min,31.73920000000000030127,0.00000000000000000000,50.82529999999999859028,47.66460000000000007958,97.24550000000000693490,-142.22381362991603737100,364.00000000000000000000
25%,43.87420000000000186446,0.00000000000000000000,70.47204999999999586180,99.08865000000000122782,217.36394999999998844942,-108.33179153723506260576,379.39999999999997726263
50%,45.40840000000000031832,0.00000000000000000000,78.90760000000000218279,133.66630000000000677574,284.34080000000000154614,-78.22634730407472147817,399.80000000000001136868
75%,46.13455000000000438831,0.00000000000000000000,89.76980000000000359250,187.03264999999998963176,339.84704999999996744009,-18.03439498010151353924,409.60000000000002273737
max,52.55989999999999895408,29.81259999999999976694,116.20829999999999415650,315.28780000000000427463,394.87950000000000727596,111.80265154555662832081,420.00000000000000000000


In [8]:
train_xy = pd.concat([train_x,train_y],axis=1)
test_xy = pd.concat([test_x,test_y],axis=1)
setup(data=train_xy,target = 'DSL D-95', test_data = test_xy,  silent = True,session_id=3973)

,Description,Value
0,Session id,3973
1,Target,DSL D-95
2,Target type,regression
3,Data shape,"(2148, 7)"
4,Train data shape,"(2148, 7)"
5,Test data shape,"(699, 7)"
6,Numeric features,6
7,Preprocess,True
8,Imputation type,simple
9,Numeric imputation,mean


In [12]:
bm = compare_models(sort = 'mae')

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
omp,Orthogonal Matching Pursuit,8.7421,134.3231,10.2381,-1.7071,0.0259,0.0223,0.0070
dummy,Dummy Regressor,9.0839,139.7785,10.4899,-1.8166,0.0265,0.0231,0.0050
llar,Lasso Least Angle Regression,9.0839,139.7785,10.4899,-1.8166,0.0265,0.0231,0.0070
lasso,Lasso Regression,9.2506,150.4070,10.6765,-1.6805,0.0270,0.0235,0.1670
en,Elastic Net,9.2617,151.0063,10.6928,-1.6843,0.0270,0.0235,0.0060
br,Bayesian Ridge,9.2762,151.1709,10.7009,-1.6919,0.0270,0.0236,0.0060
lr,Linear Regression,9.2776,151.8381,10.7170,-1.6897,0.0271,0.0236,0.4410
ridge,Ridge Regression,9.2776,151.8380,10.7170,-1.6897,0.0271,0.0236,0.0060
lar,Least Angle Regression,9.2776,151.8381,10.7170,-1.6897,0.0271,0.0236,0.0060
knn,K Neighbors Regressor,10.0078,192.1958,12.8835,-6.3412,0.0327,0.0254,0.0080


In [13]:
cm = create_model('xgboost')

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,7.6049,98.9503,9.9474,-3.3177,0.0251,0.0191
1,9.2020,176.5082,13.2856,-0.4042,0.0340,0.0238
2,11.4509,203.9884,14.2825,-22.9309,0.0361,0.0283
3,5.4812,60.4448,7.7746,0.2688,0.0196,0.0139
4,6.4417,65.6478,8.1023,-0.3102,0.0204,0.0160
5,6.4429,76.5625,8.7500,-3.6349,0.0218,0.0159
6,9.7515,151.0167,12.2889,-6.1409,0.0313,0.0243
7,12.8240,195.4769,13.9813,-0.1915,0.0350,0.0319
8,16.5033,349.0370,18.6825,-1.0627,0.0465,0.0402
9,20.4788,481.5923,21.9452,-8.9455,0.0563,0.0543


In [14]:
y_pred = predict_model(cm, data = test_x)
from sklearn.metrics import mean_absolute_error
score = mean_absolute_error(test_y,y_pred['Label'])
score

0.8846006554424857

In [57]:
loaded_model = load_model('0_88_model')
y_pred = predict_model(loaded_model, data = test_x)
from sklearn.metrics import mean_absolute_error
score = mean_absolute_error(test_y,y_pred['Label'])
score

INFO:logs:Initializing load_model()
INFO:logs:load_model(model_name=0_88_model, platform=None, authentication=None, verbose=True)
INFO:logs:Initializing predict_model()
INFO:logs:predict_model(estimator=Pipeline(memory=Memory(location=C:\Users\ineeji\AppData\Local\Temp\joblib),
         steps=[('placeholder', None),
                ['trained_model',
                 XGBRegressor(base_score=None, booster='gbtree', callbacks=None,
                              colsample_bylevel=None, colsample_bynode=None,
                              colsample_bytree=None, early_stopping_rounds=None,
                              enable_categorical=False, eval_metric=None,
                              feature_types=None, gamma=None, gpu...
                              grow_policy=None, importance_type=None,
                              interaction_constraints=None, learning_rate=None,
                              max_bin=None, max_cat_threshold=None,
                              max_cat_to_onehot=

Transformation Pipeline and Model Successfully Loaded


0.8846006554424857